# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Решение:

## 1. Основные метрики

In [2]:
import pandas as pd

In [406]:
horses = pd.read_csv('horse_data.csv', header=None, na_values='?')
horses.columns = ['surgery', 'age', 'hospital_number', 'rectal_temperature', 'pulse', 'respiratory_rate', 'temperature_of_extremities', 'peripheral_pulse', 'mucous_membranes', 'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distension', 'nasogastric_tube', 'nasogastric_reflux', 'nasogastric_reflux_PH', 'rectal_examination_-_feces', 'abdomen', 'packed_cell_volume', 'total_protein', 'abdominocentesis_appearance', 'abdomcentesis_total_protein', 'outcome', 'surgical_lesion', 'type_of_lesion_1', 'type_of_lesion_2', 'type_of_lesion_3', 'cp_data']
horses = horses[['surgery', 'age', 'rectal_temperature', 'pulse', 'respiratory_rate', 'peripheral_pulse',  'pain', 'outcome']]
horses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery             299 non-null    float64
 1   age                 300 non-null    int64  
 2   rectal_temperature  240 non-null    float64
 3   pulse               276 non-null    float64
 4   respiratory_rate    242 non-null    float64
 5   peripheral_pulse    231 non-null    float64
 6   pain                245 non-null    float64
 7   outcome             299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [61]:
horses.describe()

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,231.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.017316,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.042428,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,2.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [67]:
horses.mode()

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
0,1.0,1,38.0,48.0,20.0,1.0,3.0,1.0


In [66]:
horses.median()

surgery                1.0
age                    1.0
rectal_temperature    38.2
pulse                 64.0
respiratory_rate      24.5
peripheral_pulse       2.0
pain                   3.0
outcome                1.0
dtype: float64

**Операция**, **возраст**, **итог**, **периферический пульс** – номинальные столбцы, расчет метрик для них не имеет большого значения.
Единственное, что можно отметить (по среднему арифметическому):
* 40% наблюдаемых лошадей перенесли операцию,
* более 55% наблюдаемых лошадей выжили,
* большая часть лошадей — взрослые (среднее арифметическое значительно ближе к 1, чем к 9).

**Боль:**
Тоже качественный столбец. Сложно точно сказать порядковое это значение или номинальное, но я склонен скорее отнести его к порядковым (чем выше значение — тем хуже состояние).
* Медиана, мода и среднее арифметическое близки к среднему значение диапазона, а величина среднеквадратичного отклоения довольно велика, что позволяет сказать, что значения почти равномерно распределены по всему диапазону.

**Температура:**
* У 3/4 лошадей повышенная температура (нормальная температура — 37.8 находится на первой квартили),
* При этом у большинства лошадей отклонение от нормальной температуры — не больше 1 градуса,
* Судя по относительно небольшой величине среднеквадратичного отклонения, температура большинства лошадей близка с средней, слегка повышенной (38.2). Значительные отклонения (-2.4, +3.0) — возможно, выбросы. Проверим на следующем шаге.
* Размах температуры выглядит относительно небольшим (5.4 градуса), но минимальное и максимальное значение близки к смертельным, так что фактически наш размах близок к максимально возможному.

**Пульс:**
* Более чем у 3/4 лошадей повышенный пульс. Даже значение на первой квартили — выше нормального.
* Максимальное значение — больше чем в два раза выше значения на третьей квартили. Сразу заметно, что это выброс.

**Частота дыхания:**
* Абсолютно то же самое, что и с пульсом. Нормальное значение — минимальное. У подавляющего большинства лошадей повышенный пульс. Максимальное значение — явный выброс.

## 2. Выбросы

### 2.1. Категориальные столбцы

В категориальных столбцах — **Операция**, **Возраст**, **Итог**,  **Боль**,  **Периферический пульс** — выбросы искать нет смысла, ищем в количественных столбцах: **Пульс**, **Температура**, **Частота дыхания**.

### 2.2. Пульс

In [171]:
q1 = horses.pulse.quantile(0.25)
q3 = horses.pulse.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horses[horses.pulse.between(lower_bound, upper_bound, inclusive=True)]
remove_outliers.describe()

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
count,287.000000,287.000000,235.000000,287.000000,237.000000,229.000000,241.000000,287.000000
mean,1.407666,1.473868,38.153191,70.364934,29.751055,2.017467,2.941909,1.550523
std,0.492259,1.891788,0.732532,25.584763,16.842027,1.042791,1.308922,0.745657
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.100000,64.000000,24.000000,2.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.000000,40.800000,146.000000,96.000000,4.000000,5.000000,3.000000


In [30]:
horses[horses.pulse.lt(lower_bound)]

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,peripheral_pulse,mucous_membranes,capillary_refill_time,...,packed_cell_volume,total_protein,abdominocentesis_appearance,abdomcentesis_total_protein,outcome,surgical_lesion,type_of_lesion_1,type_of_lesion_2,type_of_lesion_3,cp_data


In [64]:
horses[horses.pulse.gt(upper_bound)]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
3,1.0,9,39.1,164.0,84.0,1.0,2.0,2.0
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0
55,1.0,9,38.6,160.0,20.0,NaN,3.0,2.0
255,1.0,9,38.8,184.0,84.0,NaN,4.0,2.0
275,1.0,9,38.8,150.0,50.0,3.0,5.0,2.0


**Вывод**: В выбросы попали пять записей и все они — жеребята (младше 6 месяцев). Можно предположить, что в данных нет ошибок, просто организм жеребят сильнее реагирует на болезнь. Четверо из пяти жеребят умерли, что также косвенно подтверждает предположение, что в данных нет ошибки. Попадание в выбросы может быть обусловлено еще и тем, что жеребят в нашей выборке значительно меньше, чем взрослых лошадей (всего 8%).

Дальнейшая работа с этими выбросами зависит от целей исследования. При расчете метрик по всей выборке я бы их игнорировал, поскольку они относятся к специфической группе данных. Но при расчетах, например, метрик отдельно для жеребят я бы использовал эти данные. 

### 2.3 Температура

In [206]:
q1 = horses.rectal_temperature.quantile(0.25)
q3 = horses.rectal_temperature.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers_temp = horses[horses.rectal_temperature.between(lower_bound, upper_bound, inclusive=True)]
remove_outliers_temp.describe()

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
count,226.00000,226.000000,226.000000,226.000000,199.000000,178.000000,187.000000,226.000000
mean,1.40708,1.672566,38.169027,69.172566,30.170854,1.814607,2.759358,1.473451
std,0.49238,2.224879,0.571949,28.394230,18.425963,0.988299,1.236043,0.719076
min,1.00000,1.000000,36.800000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.00000,1.000000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.00000,1.000000,38.200000,60.000000,24.000000,1.000000,3.000000,1.000000
75%,2.00000,1.000000,38.500000,84.000000,36.000000,3.000000,4.000000,2.000000
max,2.00000,9.000000,39.500000,184.000000,96.000000,4.000000,5.000000,3.000000


In [133]:
horses[horses.rectal_temperature.lt(lower_bound)]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0
80,1.0,1,36.4,98.0,35.0,3.0,4.0,2.0
118,1.0,1,36.5,78.0,30.0,NaN,5.0,1.0
141,2.0,1,36.0,42.0,30.0,NaN,NaN,2.0
238,2.0,1,36.1,88.0,NaN,3.0,3.0,3.0
251,2.0,1,36.6,42.0,18.0,3.0,1.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [134]:
horses[horses.rectal_temperature.gt(upper_bound)]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0
54,2.0,1,40.3,114.0,36.0,3.0,2.0,3.0
75,1.0,9,39.7,100.0,NaN,3.0,2.0,3.0
91,2.0,1,40.3,114.0,36.0,3.0,2.0,2.0
99,2.0,1,39.6,108.0,51.0,3.0,2.0,1.0
259,1.0,1,40.8,72.0,42.0,3.0,2.0,2.0
281,2.0,1,40.0,78.0,NaN,3.0,2.0,2.0


In [177]:
horses.outcome.value_counts('.')

1.0    0.595318
2.0    0.257525
3.0    0.147157
Name: outcome, dtype: float64

In [178]:
remove_outliers_temp.outcome.value_counts('.')

1.0    0.659292
2.0    0.207965
3.0    0.132743
Name: outcome, dtype: float64

**Вывод:**
С одной стороны нужно отметить, что температуры выбросов, хотя и значительно отличаются он нормальных (и средних) значений, все-таки реальны. У лошадей могли быть такие температуры. С другой — большинство лошадей, попавших в выбросы (10 из 14), не выжили. Вероятнее всего, ошибок в данных нет, просто это термильнальные случаи, когда шансы врачей спасти лошадь были минимальными.

При удалении выбросов на 6,4% снижается уровень смертности. Мне кажется, это довольно серьезное искажение исходных данных. Поэтому я бы не удалял эти записи. При необходимости можно заменить их значениями верхней и нижней границы.

### 2.4 Частота дыхания

In [72]:
q1 = horses.respiratory_rate.quantile(0.25)
q3 = horses.respiratory_rate.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers_resp = horses[horses.respiratory_rate.between(lower_bound, upper_bound, inclusive=True)]
remove_outliers_resp.describe()

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
count,224.000000,225.000000,195.000000,219.000000,225.000000,176.000000,191.000000,224.000000
mean,1.424107,1.426667,38.118462,68.716895,26.715556,1.948864,2.890052,1.540179
std,0.495314,1.801587,0.726287,26.245178,11.461464,1.043455,1.286961,0.738555
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.750000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.100000,60.000000,24.000000,1.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,86.000000,35.000000,3.000000,4.000000,2.000000
max,2.000000,9.000000,40.800000,160.000000,60.000000,4.000000,5.000000,3.000000


In [73]:
horses[horses.respiratory_rate.gt(upper_bound)]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
3,1.0,9,39.1,164.0,84.0,1.0,2.0,2.0
39,1.0,9,39.2,146.0,96.0,NaN,NaN,2.0
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0
82,1.0,9,38.1,100.0,80.0,1.0,3.0,1.0
84,1.0,1,37.8,60.0,80.0,3.0,2.0,1.0
103,1.0,9,38.0,140.0,68.0,1.0,3.0,1.0
106,1.0,1,38.3,52.0,96.0,3.0,NaN,1.0
120,1.0,1,39.4,54.0,66.0,1.0,2.0,1.0
125,1.0,1,38.0,42.0,68.0,1.0,3.0,1.0
186,1.0,1,39.3,64.0,90.0,3.0,NaN,1.0


In [74]:
horses[horses.respiratory_rate.lt(lower_bound)]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome


**Вывод**: значительная часть выбросов — жеребята, у которых, как мы уже заметили параметры могут сильно отлиаться от взрослых лошадей. Работа с этими записями зависит от того, что нас больше интересует: жеребята или взрослые лошади. Если нас интересует выборка в целом, я бы удалил этих жеребят. Если нас интересуют жеребята, эти записи нужно оставить.

Для остальных записей найти столько-нибудь понятные закономерности или зависимости от других признаков не удалось. Поэтому их я бы заменил значениями верхней границы.

## 3. Пропуски

In [407]:
horses_initial = horses.copy()

### 3.1. Операция

In [408]:
horses.loc[horses['surgery'].isna()]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
132,NaN,1,38.0,48.0,20.0,1.0,4.0,NaN


В столбце «surgery» всего один пропуск. По вторичным признакам можно предположить, что операция была проведена.
* Тут я решил воспользоваться дополнительным столбцом «surgical lesion» из первоначального набора. Подавляющему большинству лошадей (82%), у которых, как и у этой лошади, «surgical lesion» == 1, проводили операцию. Из тех, у кого «surgical lesion» == 2, 79% обошились без операции.
* Уровень боли (pain) – 4. В описании указано: чем выше уровень боли, тем выше вероятность операции. 28 из 38 лошадей с таким уровнем боли были прооперированы.

Поэтому заменяем этот пропуск на 1 (была операция).

In [409]:
horses.loc[horses['surgery'].isna(), 'surgery'] = 1

### 3.2. Итог

Пропуск в столбце **Итог** всего один. У той же лошади, что и в первом пункте. Ее основные показатели (температура, дыхание, пульс) близки к нормальным. Поэтому предполагаем, что она выжила. Это не должно сильно сказаться на результатах, а терять данные ни к чему.

In [410]:
horses.loc[horses['outcome'].isna(), 'outcome'] = 1

### 3.3. Возраст

В этом столбце все хорошо, пропусков нет.

### 3.4. Отбрасываем стобцы, где почти нет данных

У нас уже есть три столбца без пропусков. При этом строки, которые содержат только данные о возрасте, операции и итоге, практически не несут информации (только если мы хотим установить зависимость смертности от проведенной операции). В остальном они почти бесполезны (мы ничего не знаем о состоянии лошади). Поэтому мы отбрасываем все строки, где заполнено меньше четырех полей.

In [411]:
horses.dropna(thresh=4, inplace=True)
horses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 0 to 299
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery             292 non-null    float64
 1   age                 292 non-null    int64  
 2   rectal_temperature  240 non-null    float64
 3   pulse               276 non-null    float64
 4   respiratory_rate    242 non-null    float64
 5   peripheral_pulse    231 non-null    float64
 6   pain                245 non-null    float64
 7   outcome             292 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 20.5 KB


In [266]:
horses.outcome.value_counts('.')

1.0    0.595890
2.0    0.253425
3.0    0.150685
Name: outcome, dtype: float64

**Вывод:** Удаление этих выбросов почти не повлияло на распределение выживших/погибших лошадей. Значит, все хорошо. Мы не исказили данные.

### 3.5. Пульс

Для всех лошадей (за исключением одной) у которых не указан пульс, есть данные о периферическом пульсе. Мы можем использовать эту информацию. Можно заполнить данные о пульсе средним значением пульса у лошадей с таким же периферическим пульсом. Поскольку, как мы уже знаем, у жеребят наблюдаются значительные отклонения пульса, а все лошади из нашего списка взрослые, при заполнении пропусков мы будем использовать только данные взрослых лошадей.

In [244]:
horses[horses.pulse.isna()]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
5,2.0,1,NaN,NaN,NaN,1.0,2.0,1.0
52,2.0,1,NaN,NaN,NaN,1.0,1.0,1.0
58,1.0,1,NaN,NaN,20.0,3.0,5.0,2.0
78,1.0,1,NaN,NaN,NaN,3.0,5.0,2.0
83,1.0,1,38.0,NaN,24.0,3.0,5.0,2.0
93,2.0,1,NaN,NaN,NaN,3.0,5.0,2.0
115,2.0,1,NaN,NaN,40.0,1.0,3.0,1.0
117,2.0,1,39.5,NaN,NaN,3.0,3.0,3.0
126,1.0,1,38.7,NaN,NaN,1.0,5.0,1.0
136,1.0,1,NaN,NaN,NaN,3.0,5.0,2.0


In [412]:
horses.loc[(horses.pulse.isna()) & (horses.peripheral_pulse == 1), 'pulse'] = round(horses.loc[(horses.peripheral_pulse == 1) & (horses.age == 1)].pulse.mean())
horses.loc[(horses.pulse.isna()) & (horses.peripheral_pulse == 2), 'pulse'] = round(horses.loc[(horses.peripheral_pulse == 2) & (horses.age == 1)].pulse.mean())
horses.loc[(horses.pulse.isna()) & (horses.peripheral_pulse == 3), 'pulse'] = round(horses.loc[(horses.peripheral_pulse == 3) & (horses.age == 1)].pulse.mean())
horses.loc[(horses.pulse.isna()) & (horses.peripheral_pulse == 4), 'pulse'] = round(horses.loc[(horses.peripheral_pulse == 4) & (horses.age == 1)].pulse.mean())

Единственной лошади, значение периферического пульса которой неизвестно, заполняем пропуск средним арифметическим (заполнение медианой приводит к более значительному изменению основных метрик)

In [413]:
horses.pulse.fillna(round(horses[horses.age == 1].pulse.mean()), inplace=True)

In [414]:
horses.pulse.describe()

count    292.000000
mean      71.928082
std       28.045955
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

**Вывод:** Заполнение пропусков практически не повлияло на основные метрики. Среднеквадратичное отклонение поменялось на десятые доли, медиана — на сотые. Остальные метрики не изменились.

### 3.6. Температура

In [221]:
horses[horses.rectal_temperature.isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 5 to 295
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery             52 non-null     float64
 1   age                 52 non-null     int64  
 2   rectal_temperature  0 non-null      float64
 3   pulse               52 non-null     float64
 4   respiratory_rate    32 non-null     float64
 5   peripheral_pulse    41 non-null     float64
 6   pain                45 non-null     float64
 7   outcome             52 non-null     float64
dtypes: float64(7), int64(1)
memory usage: 3.7 KB


In [252]:
horses.rectal_temperature.describe()

count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64

Смотрим, что произойдет при заполнении средним:

In [270]:
horses.rectal_temperature.fillna(round(horses.rectal_temperature.mean(), 1)).describe()

count    292.000000
mean      38.173630
std        0.663757
min       35.400000
25%       37.875000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64

Смотрим, что произойдет при заполнении медианой:

In [269]:
horses.rectal_temperature.fillna(round(horses.rectal_temperature.median(), 1)).describe()

count    292.000000
mean      38.173630
std        0.663757
min       35.400000
25%       37.875000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64

In [254]:
horses.rectal_temperature.mode()

0    38.0
dtype: float64

In [228]:
horses.rectal_temperature.median()

38.2

При заполнении медианой метрики искажаются меньше:

In [415]:
horses.rectal_temperature.fillna(round(horses.rectal_temperature.median(), 1), inplace=True)

In [357]:
horses.rectal_temperature.mode()

0    38.2
dtype: float64

In [273]:
horses.rectal_temperature.median()

38.2

**Вывод:** Поскольку проследить сколько-нибудь значимую зависимость температуры от других параметров не получилось, заполняем пропуски медианой. При этом ожидаемо и довольно ощутимо изменилось среднеквадратичное отклонение и поменялась мода. Но приходится чем-то жертовать. Остальные метрики не изменились или изменились почти незначительно.

### 3.7. Частота дыхания

In [297]:
horses[horses.respiratory_rate.isna()]

,surgery,age,rectal_temperature,pulse,respiratory_rate,peripheral_pulse,pain,outcome
5,2.0,1,38.2,54.0,NaN,1.0,2.0,1.0
7,1.0,1,38.2,60.0,NaN,NaN,NaN,2.0
9,2.0,9,38.3,90.0,NaN,NaN,5.0,1.0
29,2.0,1,37.7,48.0,NaN,1.0,1.0,1.0
32,1.0,1,37.2,60.0,NaN,1.0,3.0,1.0
37,1.0,1,37.8,72.0,NaN,3.0,5.0,1.0
38,2.0,1,38.6,52.0,NaN,1.0,3.0,1.0
40,1.0,1,38.2,88.0,NaN,3.0,5.0,2.0
43,1.0,1,38.2,120.0,NaN,4.0,4.0,3.0
45,2.0,1,38.2,120.0,NaN,3.0,5.0,2.0


In [379]:
horses.loc[(horses.age == 9) & (horses.respiratory_rate.isna()), 'respiratory_rate'] = round(horses[horses.age == 9].respiratory_rate.median())

In [380]:
horses.respiratory_rate.fillna(round(horses.respiratory_rate.median()), inplace=True)

In [381]:
horses.respiratory_rate.describe()

count    292.000000
mean      29.818493
std       16.217315
min        8.000000
25%       20.000000
50%       26.000000
75%       36.000000
max       96.000000
Name: respiratory_rate, dtype: float64

**Вывод:** Все примерно так же, как и в предыдущем пункте, за одним исключением. В нашей подборке оказались несколько жеребят с высокой температурой и пульсом. Поскольку жеребятам в целом своийственны более тяжелые симптомы, данные по ним мы заполняем не общей медианой, а медианой по жеребятам. При этом основные метрики меняются меньше, чем при простом заполнении медианой.

### 3.8. Периферический пульс

In [382]:
horses.peripheral_pulse.value_counts('.')

1.0    0.497835
3.0    0.445887
4.0    0.034632
2.0    0.021645
Name: peripheral_pulse, dtype: float64

In [363]:
horses[horses.peripheral_pulse.isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1 to 299
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery             60 non-null     float64
 1   age                 60 non-null     int64  
 2   rectal_temperature  60 non-null     float64
 3   pulse               60 non-null     float64
 4   respiratory_rate    60 non-null     float64
 5   peripheral_pulse    0 non-null      float64
 6   pain                32 non-null     float64
 7   outcome             60 non-null     float64
dtypes: float64(7), int64(1)
memory usage: 4.2 KB


У нас есть данные пульса. Посмотрим, как связаны пульс и периферический пульс.

In [454]:
pp1 = horses[horses.peripheral_pulse == 1].pulse.median()
pp1

48.5

In [330]:
pp2 = horses[horses.peripheral_pulse == 2].pulse.median()
pp2

60.0

In [341]:
pp3 = horses[horses.peripheral_pulse == 3].pulse.median()
pp3

85.0

In [335]:
pp2 = horses[horses.peripheral_pulse == 4].pulse.median()
pp2

89.5

Хотя прямой зависимости между ними нет, мы видим, что в среднем чем выше периферический пульс, тем выше пульс. Попробуем использовать это при заполнении пропусков.

In [432]:
horses.loc[(horses.peripheral_pulse.isna()) & (horses.pulse <= pp1), 'peripheral_pulse'] = 1
horses.loc[(horses.peripheral_pulse.isna()) & (horses.pulse <= pp2), 'peripheral_pulse'] = 2
horses.loc[(horses.peripheral_pulse.isna()) & (horses.pulse <= pp3), 'peripheral_pulse'] = 3
horses.loc[(horses.peripheral_pulse.isna()) & (horses.pulse > pp3), 'peripheral_pulse'] = 4

In [433]:
horses.peripheral_pulse.describe()

count    292.000000
mean       2.030822
std        1.062841
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        4.000000
Name: peripheral_pulse, dtype: float64

In [424]:
horses_initial.peripheral_pulse.describe()

count    231.000000
mean       2.017316
std        1.042428
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        4.000000
Name: peripheral_pulse, dtype: float64

In [429]:
horses_initial.peripheral_pulse.fillna(horses_initial.peripheral_pulse.mode()[0]).describe()

count    300.000000
mean       1.783333
std        1.009846
min        1.000000
25%        1.000000
50%        1.000000
75%        3.000000
max        4.000000
Name: peripheral_pulse, dtype: float64

In [435]:
horses.peripheral_pulse.value_counts('.')

1.0    0.472603
3.0    0.352740
2.0    0.099315
4.0    0.075342
Name: peripheral_pulse, dtype: float64

In [434]:
horses_initial.peripheral_pulse.value_counts('.')

1.0    0.497835
3.0    0.445887
4.0    0.034632
2.0    0.021645
Name: peripheral_pulse, dtype: float64

In [437]:
horses_initial.peripheral_pulse.fillna(horses_initial.peripheral_pulse.mode()[0]).value_counts('.')

1.0    0.613333
3.0    0.343333
4.0    0.026667
2.0    0.016667
Name: peripheral_pulse, dtype: float64

**Вывод**: Алгоритм заполнения безусловно не идеальный. Хотя основные метрики меняются не слишком сильно, рапределение, конечно, нарушается, что особенно заметно на value_counts. Но такие результаты все-таки лучше, чем при заполнении модой.

### 3.9. Боль

Связать боль с другими параметрами сложно, но заполнять медианой было бы совсем странно. Поэтому попробуем посмотреть на связь боли с конечным исходом (выжила лошадь или умерла)

In [440]:
horses[horses.outcome == 1].pain.value_counts()

3.0    54
1.0    35
2.0    33
5.0    12
4.0    11
Name: pain, dtype: int64

In [441]:
horses[horses.outcome == 2].pain.value_counts()

5.0    24
4.0    17
2.0    14
3.0     8
1.0     2
Name: pain, dtype: int64

In [442]:
horses[horses.outcome == 3].pain.value_counts()

2.0    12
4.0    11
5.0     6
3.0     5
1.0     1
Name: pain, dtype: int64

Связь уровня боли с исходом прослеживается. Поэтому попробуем отдельно заполнить пропуски для каждого значения outcome

In [448]:
horses.loc[(horses.pain.isna()) & (horses.outcome == 1), 'pain'] = horses[horses.outcome == 1].pain.mode()[0]
horses.loc[(horses.pain.isna()) & (horses.outcome == 2), 'pain'] = horses[horses.outcome == 2].pain.mode()[0]
horses.loc[(horses.pain.isna()) & (horses.outcome == 3), 'pain'] = horses[horses.outcome == 3].pain.mode()[0]

In [449]:
horses.pain.describe()

count    292.000000
mean       2.989726
std        1.261332
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: pain, dtype: float64

In [450]:
horses_initial.pain.describe()

count    245.00000
mean       2.95102
std        1.30794
min        1.00000
25%        2.00000
50%        3.00000
75%        4.00000
max        5.00000
Name: pain, dtype: float64

In [451]:
horses.pain.value_counts()

3.0    96
2.0    68
5.0    51
4.0    39
1.0    38
Name: pain, dtype: int64

In [452]:
horses_initial.pain.value_counts()

3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64

**Вывод:** распределение, безусловно, изменилось, но основные метрики изменились не слишком значительно. Такое решение выглядить более осмысленным, чем заполнение модой или медианой.